# Import necessory packages

In [ ]:
#import ezdxf
import os
import time
import pandas as pd
import re
#import win32com.client

# Extract text from Autocad

## 天正引出标注

In [55]:

for layout in layouts:
    for entity in layout.Block:
        name = entity.EntityName
        if name == 'TDbSymbMultiLeader':
            print(entity.UpText,entity.DownText) #上标文字，下标文字


5 6
7 8


## 天正索引

In [23]:
for layout in layouts:
    for entity in layout.Block:
        name = entity.EntityName
        if name == 'TDbSymbIndexPointer':
            print(entity.UpText,entity.DownText,entity.IndexLabel,entity.DrawingLabel) #上标文字，下标文字，索引编号，图号

1 2 3 4


## 天正做法标注

In [62]:
for layout in layouts:
    for entity in layout.Block:
        name = entity.EntityName
        if name == 'TDbSymbComposing' or name == 'TDbText':
            print(entity.Text) #文字内容

一
二
三
天正单行文字输入


## Autocad多行及单行文字 

In [51]:
for layout in layouts:
    for entity in layout.Block:
        name = entity.EntityName
        if name == 'AcDbText' or name == 'AcDbMText' or name == 'AcDbMLeader':
            print(entity.TextString) #文字内容

多行文字
Autocad文字
多重引线\P多重引线第二行
\A1;引线\P引线第二行


## 天正单行文字 

In [45]:
for layout in layouts:
    for entity in layout.Block:
        name = entity.EntityName
        if name == 'TDbText':
            print(entity.Text) #文字内容

天正单行文字输入


## cheking the Entity.Name from dwg file

In [61]:
for layout in layouts:
    for entity in layout.Block:
        name = entity.EntityName
        print(name)

TDbSymbIndexPointer
TDbSymbMultiLeader
TDbSymbComposing
TDbSymbIndexPointer
TDbText
AcDbMText
AcDbText
AcDbMLeader
AcDbLeader
AcDbMText
TDbSymbDrawingIndex
TDbDrawingName
TDbSymbArrow
AcDbViewport
TDbSymbMultiLeader


## Exztracting with win32com(Pyautocad is much faster)

In [160]:
# acad = win32com.client.Dispatch("AutoCAD.Application")
# extracted_text = []  # Local variable to store extracted text

In [172]:
# def extract_text_from_dwg(dwg_file):
#     try:
#         doc = acad.Documents.Open(dwg_file)
#         time.sleep(3)
#         # Access layouts through the 'Layouts' property of the 'Application' object
#         layouts = acad.Application.Documents.Item(doc.Name).Layouts
#         for layout in layouts:
#             print('Extracting text from layout:',layout.Name)
#             for entity in layout.Block:
#                 name = entity.EntityName
#                 if name == 'AcDbText' or name == 'AcDbMText' or name == 'AcDbMLeader':
#                     extracted_text.append(entity.TextString)
#                 if name == 'TDbSymbMultiLeader':
#                     extracted_text.append([entity.UpText,entity.DownText])
#                 if name == 'TDbSymbIndexPointer':
#                     extracted_text.append([entity.UpText,entity.DownText]) #上标文字，下标文字
#                 if name == 'TDbSymbComposing'or name == 'TDbText':
#                     extracted_text.append(entity.Text) #文字内容
#         return extracted_text
#     except Exception as e:
#         return str(e)

## using Pyautocad to extract material from dwg file

In [2]:
# Set autocad environment and set an empty list to store material name
from pyautocad import Autocad
acad = Autocad(create_if_not_exists=True)

In [3]:
# Define the function to extract material from the drawing
def extract_text_from_dwg(dwg_file):
    try:    
        acad = Autocad(create_if_not_exists=True)
        doc = acad.app.Documents.Open(dwg_file)  
        time.sleep(1)    
        for i in range(doc.Layouts.Count):
            layout = doc.Layouts[i]
            print('extracting from layout: %s' % layout.Name)
            for entity in acad.iter_objects(['AcDbText','AcDbMText','AcDbMLeader','TDbSymbMultiLeader','TDbSymbIndexPointer','TDbSymbComposing','TDbText'],block=layout.Block,dont_cast=True):
                name = entity.ObjectName
                if name == 'AcDbText' or name == 'AcDbMText' or name == 'AcDbMLeader':
                    extracted_text.append(entity.TextString)
                if name == 'TDbSymbMultiLeader':
                    extracted_text.append([entity.UpText,entity.DownText])
                if name == 'TDbSymbIndexPointer':
                    extracted_text.append([entity.UpText,entity.DownText]) #上标文字，下标文字
                if name == 'TDbSymbComposing'or name == 'TDbText':
                    extracted_text.append(entity.Text) #文字内容
        doc.Close()
        return extracted_text        
    except Exception as e:
        return str(e)


In [4]:
# Define the function to find all dwg files in the folder
def find_dwg_files(file_path):
    dwg_files = []
    for root, dirs, files in os.walk(file_path):
        for file in files:
            if file.endswith(".dwg"):
                dwg_files.append(os.path.join(root, file))
    return dwg_files

In [5]:
# Set the path to the folder where the dwg files are located,and find all the dwg files in the folder
file_path = r'W:\城建院数据\20220526 西藏项目\05-施工图纸\中心公园\详图'
dwg_files = find_dwg_files(file_path)
dwg_files

['W:\\城建院数据\\20220526 西藏项目\\05-施工图纸\\中心公园\\详图\\XS-01中心公园-做法详图0322（改沥青厚度0706）.dwg',
 'W:\\城建院数据\\20220526 西藏项目\\05-施工图纸\\中心公园\\详图\\XS-02LOGO字详图-s.dwg',
 'W:\\城建院数据\\20220526 西藏项目\\05-施工图纸\\中心公园\\详图\\XS-03040506设施详图-wsd-s.dwg',
 'W:\\城建院数据\\20220526 西藏项目\\05-施工图纸\\中心公园\\详图\\XS-0710书写互动亭详图+弧形廊架-20220325-s.dwg',
 'W:\\城建院数据\\20220526 西藏项目\\05-施工图纸\\中心公园\\详图\\XS-08张拉膜-s.dwg',
 'W:\\城建院数据\\20220526 西藏项目\\05-施工图纸\\中心公园\\详图\\XS-1314指示牌标识牌详图-20220325-s.dwg']

In [6]:
#Empty list to store extracted text
extracted_text = []
# Extract all text from dwg files in a folder
for dwg_file in dwg_files:
    try:
        t1 = time.time()
        print('Opening file: ' + dwg_file)
        extract_text_from_dwg(dwg_file)
        t2 = time.time()
        print(f'Time taken: ' + str(t2-t1) + ' seconds')
    except:
        print('Error in file: ' + dwg_file)
        pass
extracted_text

Opening file: W:\城建院数据\20220526 西藏项目\05-施工图纸\中心公园\详图\XS-01中心公园-做法详图0322（改沥青厚度0706）.dwg
extracting from layout: Model
Time taken: 197.41223454475403 seconds
Opening file: W:\城建院数据\20220526 西藏项目\05-施工图纸\中心公园\详图\XS-02LOGO字详图-s.dwg


In [368]:
len(extracted_text)

4934

In [369]:
# defin a function to clean the text extracted from autocad
def clean_text(text):
    cleaned_text = []
    for item in extracted_text:
        if isinstance(item, list):
            cleaned_text.append(item[0])
            cleaned_text.append(item[1])
        elif '\\P' in item:
            for i in range(len(item.split('\\P'))):
                cleaned_text.append(item.split('\\P')[i])
        elif '\r\n' in item:
            for i in range(len(item.split('\r\n'))):
                cleaned_text.append(item.split('\r\n')[i])
        else:
            cleaned_text.append(item)
    cleaned_text = list(map(lambda x: x.replace('\\A1;', ''),cleaned_text))
    return cleaned_text

In [370]:
cleaned_text = clean_text(extracted_text)
cleaned_text

['',
 '<1:2',
 '<1:2',
 '注：图中标注尺寸为',
 'mm',
 '注：图中标注尺寸为',
 'mm',
 '注：图中标注尺寸为',
 'mm',
 '注：图中标注尺寸为',
 'mm',
 '注：图中标注尺寸为',
 'mm',
 '600',
 '180',
 '粒径',
 '50-80mm',
 '普通河滩卵石',
 '500',
 '厚种植土垫层',
 '二布一膜，铺设至不透水层',
 '200',
 '厚粗砂垫层',
 '素土夯实，夯实度＞',
 '0.93',
 '预制',
 'C20',
 '砼垫块',
 '常水位',
 '种植土',
 '大卵石，粒径',
 '200-400',
 '常水位',
 '400X400X100',
 '厚白麻荔枝面花岗岩汀步石',
 '20',
 '厚',
 '1:3',
 '干硬性水泥砂浆',
 '两布一膜',
 'C20',
 '钢筋砼基础，',
 '8@200',
 '单层双向',
 '素土夯实，夯实度＞',
 '0.93',
 '300',
 '100',
 '400',
 '200',
 '400',
 '30',
 '150',
 '50',
 '300',
 '300',
 '200',
 '150',
 '600',
 '200',
 '50',
 '1000',
 '120',
 '120',
 '120',
 '210',
 '120',
 '120',
 '120',
 '930',
 '600',
 '接铺装',
 '150',
 '厚',
 'C15',
 '级配砂石',
 '120',
 '厚',
 'C20',
 '砼',
 '100-200',
 '卵石',
 '30-40',
 '卵石',
 '5',
 '厚钢板收边',
 '自然土',
 '50',
 '200',
 '360',
 '150',
 '常水位',
 '200',
 '300',
 '300',
 '200',
 '3',
 '厚彩色图案涂料喷涂',
 '50',
 '厚中粒式沥青混凝土面层',
 '200',
 '厚',
 'C20',
 '砼垫层',
 '素土夯实，夯实度＞',
 '0.93',
 'C15',
 '砼垫层',
 'C20',
 '砼Φ',
 '300',
 '有筋涵管',
 'C

# Extract text from dxf files
##  You have to Batch conver the tangent entities into t3,and batch convert the dwg files into dxf files

In [371]:
# def find_dxf_files(dxf_file_path):
#     dxf_files = []
#     for root, dirs, files in os.walk(dxf_file_path):
#         for file in files:
#             if file.endswith(".dxf"):
#                 dxf_files.append(os.path.join(root, file))
#     return dxf_files

In [372]:
# dxf_file_path = r'W:\城建院数据\20220526 西藏项目\06-图纸存档\20220928-CAD-施工-滨河景观-对应0817水利版图纸全套\\'
# dxf_files = find_dxf_files(dxf_file_path)
# dxf_files

In [373]:
# def extract_text_with_ezdxf(dxf_file_path):
#     try:        
#         doc = ezdxf.readfile(dxf_file_path)
#         # for msp in doc.modelspace():
#         #     for text in msp.query('MTEXT'):
#         #         extracted_text.append(text.dxf.text)
#         for msp in doc.layouts_and_blocks():
#             for text in msp.query('MTEXT'):
#                 extracted_text.append(text.dxf.text)
#         return extracted_text
#     except Exception as e:
#         return str(e)

In [374]:
# extracted_text = []
# for dxf_file in dxf_files:    
#     print('Opening file: ' + dxf_file)
#     extract_text_with_ezdxf(dxf_file)

# Extract meaningful texts

In [375]:

# Convert the extracted text into a dataframe
df = pd.DataFrame(extracted_text)
df

,0
0,
1,<1:2
2,<1:2
3,注：图中标注尺寸为
4,mm
...,...
4929,健身器材
4930,椭圆机
4931,健身器材
4932,三联漫步机


In [376]:
for i in df[0]:
    if isinstance(i, list):
        print(i)

['接相邻铺装', '']
['接相邻铺装', '']
['100长∠75X50X3@500', '^16膨胀螺栓固定于垫层']
['5厚钢板与角钢焊接固定', '']
['通长30X30X2厚钢板,M8膨胀螺栓固定', '']
['20厚1:2.5水泥砂浆', '']
['园路、平台构造', '']
['100厚C20素砼垫层', '']
['灰土夯实（密实度≥92%）', '']
['M7.5水泥砂浆砌Mu10砖', '']
['当地自然面，灰色毛石板，', '甲方选型为准收边  直径100']
['100厚黄灰色石屑铺装，粒径3-5', '']
['1:2.5水泥砂浆卧牢', '']
['当地自然面，灰色毛石板，', '甲方选型为准收边  直径100']
['', '']
['见水生植物种植图', '']
['当地自然面，灰色毛石板，甲方选型为准收边  直径100', '']
['排水管，详水施', '']
['∠100x75x4 厂家定制', '75一侧M6膨胀螺栓与混凝凝土固定']
['∠100x75x4 厂家定制', '75一侧M6膨胀螺栓与混凝凝土固定']
['200x150通长C20素混凝土墩', '']
['见结构详图', '200厚挡墙做法']
['见水生植物种植图', '']
['当地自然面，灰色毛石板，甲方选型为准', '宽300-400mm之间，长400-600mm,高100/见平面标注']
['溢水管，详水施', '']
['见结构详图', '200厚挡墙做法']
['接相邻绿地', '']
['当地灰色毛石板汀步，自然面 石缝空隙100-200MM', '毛石边单长200-1500MM,边控制4-6边']
['毛石汀步剖面图', '详见']
['5厚钢板收边与角钢焊接固定', '']
['100厚%%c25~35本色河光卵石散铺', '']
['', '']
['30厚白麻荔枝面花岗岩碎拼，留3-5MM缝', '边数控制在4-6个 边长控制在300-600MM']
['', '']
['见水生植物种植图', '']
['换溢水沟槽，详水施', '']
['见结构详图', '200厚挡墙做法']
['大卵石 直径200-400', '']
['400x200x30厚芝麻灰花岗岩,自然面', '']
['400x400x30厚白麻荔枝面花岗

In [377]:
# Function to extract meaningful information from the string
def extract_info(s):
    # Convert list to string if necessary
    if isinstance(s, list):
        s = ' '.join(s)
    # Check if the string contains an AutoCAD font definition
    if "{" in s and "}" in s:
        # Extract font name, color code, character pitch and displayed text
        match = re.search(r'{\\f(.+?)\|b(.+?)\|i(.+?)\|c(.+?)\|p(.+?);(.+?)}', s)
        if match:
            font_name, bold, italic, color_code, character_pitch, text = match.groups()
            return pd.Series([font_name, bold, italic, color_code, character_pitch, text])
        else:
            # Extract text after "\T0.8;" if it exists
            match = re.search(r'{\\T\d+?;(.+?)}', s)
            if match:
                text = match.group(1)
                return pd.Series([None, None, None, None, None, text])
            else:
                return pd.Series([None, None, None, None, None, s])
    else:
        # Check if the string starts with "\pt" and remove it if true
        if s.startswith("\\pt"):
            s = re.sub(r'^\\pt\d+;', '', s)
        # Return the entire string
        return pd.Series([None, None, None, None, None, s])

In [378]:

# Apply the function to the dataframe
df[['font_name', 'bold', 'italic', 'color_code', 'character_pitch', 'text']] = df[0].apply(extract_info)

# Keep only the "Original Text" and "Extracted Text" columns
df_final = df[[0, 'text']].copy()
df_final.columns = ['Original Text', 'Extracted Text']

# Function to extract data in the specific format
def extract_specific_format(s):
    match = re.search(r'\d+X\d+X\d+[\u4e00-\u9fff]+', s)
    if match:
        return match.group()
    else:
        return None

# Apply the function to the "Extracted Text" column
df_final['材料标注'] = df_final['Extracted Text'].apply(extract_specific_format)
df_final = df_final.dropna(subset=['材料标注'])


In [379]:
df_final['材料标注']

184        900X300X120芝麻灰花岗岩平道牙
186                  30X30X2厚钢板
255        900X300X120芝麻灰花岗岩平道牙
352       250X250X300厚芝麻灰荔枝面花岗岩
353                250X250X300厚
                 ...           
4861     500X500X30厚芝麻灰火烧面花岗岩贴面
4867           100X500X40防腐木坐凳面
4868     500X500X30厚芝麻灰火烧面花岗岩贴面
4869    500X150X200厚白麻火烧面花岗岩平缘石
4870       300X300X40厚芝麻灰火烧面花岗岩
Name: 材料标注, Length: 455, dtype: object

In [380]:

# Save the final dataframe to a CSV file with GB2312 encoding
#df_final['材料标注'].to_csv('D:\Desktop\df_final_specific_gb2312.csv', index=False, header=False,encoding='gb2312')


In [381]:
# 创建属性和相应值的字典
attributes = {
    '规格': r'\d+X\d+X\d+',
    '颜色': ['鹤塘红', '桃花红', '番茄红', '罗源红', '虾红', '四川红', '中国红', '岑溪红', '三堡红', '贵妃红', 
          '桔红', '乳山红', '将军红', '泰山红', '石岛红', '石榴红', '阳江红', '新疆红', '天山红', '枫叶红', 
          '桂林红', '台湾红', '芝麻黑', '福鼎黑', '玄武黑', '黑金刚', '蒙古黑', '鱼鳞黑', '济南青', '中国黑', 
          '泰安绿', '高明绿', '豆绿', '浅绿', '菊花绿', '青底绿花', '浙川绿', '森林绿', '芝麻灰', '珍珠灰', 
          '鲁灰', '章丘灰', '台湾灰', '深灰', '浅灰', '芝麻白', '湖北白麻', '山东白麻', '珍珠白', '白麻', '白色', 
          '灰麻', '黄锈石', '卡拉麦里金', '菊花黄', '珍珠黄麻', '黄金麻', '黄金钻', '吉达红', '南非红', '幻彩绿', 
          '墨绿麻', '女儿红', '珊瑚红', '黑金沙', '克什米尔金', '幻彩红', '印度红', '印度金丝麻', '美国灰麻', 
          '美利坚白麻', '英国棕'],
    '面层处理': ['火烧面', '抛光面', '哑光面', '光面', '自然面', '剁斧面', '斩假面', '荔枝面', '机切面', '蘑菇面', '菠萝面', '拉丝面','烧面'],
    '材料': ['花岗岩', '防腐木', '青石板', '卵石', '烧结砖', '广场砖', '仿古砖', 'pc砖','文化石', '石材', '石英石', '大理石', '石英岩', '石英砂岩', '砂岩', '砂石', '石子',  '石英砂', '石英砂石', '石英','瓦片']
}

In [382]:

# 定义提取函数
def extract_attribute(s, attribute_values):
    if isinstance(attribute_values, list):
        for value in attribute_values:
            if value in s:
                return pd.Series([value, s.replace(value, '')])
    else:
        match = re.search(attribute_values, s)
        if match:
            value = match.group()
            return pd.Series([value, s.replace(value, '')])
    return pd.Series([None, s])

# 应用提取函数到数据帧
s = df_final['材料标注']
for attribute, values in attributes.items():
    df_final[[attribute, '其他']] = s.apply(extract_attribute, attribute_values=values)
    s = df_final['其他']
df_final = df_final.drop(columns=['Original Text', 'Extracted Text'])
df_final = df_final[['材料标注','规格','颜色','面层处理', '材料', '其他' ]]
# 保存数据帧到CSV文件
# df_final.to_csv('D:\Desktop\df_final_specific_gb2312.csv', encoding='gb2312', index=False)


In [383]:
df_final

,材料标注,规格,颜色,面层处理,材料,其他
184,900X300X120芝麻灰花岗岩平道牙,900X300X120,芝麻灰,None,花岗岩,平道牙
186,30X30X2厚钢板,30X30X2,None,None,None,厚钢板
255,900X300X120芝麻灰花岗岩平道牙,900X300X120,芝麻灰,None,花岗岩,平道牙
352,250X250X300厚芝麻灰荔枝面花岗岩,250X250X300,芝麻灰,荔枝面,花岗岩,厚
353,250X250X300厚,250X250X300,None,None,None,厚
...,...,...,...,...,...,...
4861,500X500X30厚芝麻灰火烧面花岗岩贴面,500X500X30,芝麻灰,火烧面,花岗岩,厚贴面
4867,100X500X40防腐木坐凳面,100X500X40,None,None,防腐木,坐凳面
4868,500X500X30厚芝麻灰火烧面花岗岩贴面,500X500X30,芝麻灰,火烧面,花岗岩,厚贴面
4869,500X150X200厚白麻火烧面花岗岩平缘石,500X150X200,白麻,火烧面,花岗岩,厚平缘石


# 分类整理数据

In [384]:
df_new = df_final[['材料', '颜色', '规格', '面层处理', '其他']]

In [385]:
# Group by '材料', '颜色', '规格', '面层处理' and count the frequency
summary_grouped = df_new.groupby(['材料', '颜色', '规格', '面层处理']).size().reset_index(name='频率')

summary_grouped


,材料,颜色,规格,面层处理,频率
0,花岗岩,中国黑,400X400X20,光面,3
1,花岗岩,中国黑,600X300X50,光面,4
2,花岗岩,浅灰,1000X200X40,火烧面,2
3,花岗岩,浅灰,200X100X40,火烧面,2
4,花岗岩,浅灰,200X200X40,荔枝面,6
...,...,...,...,...,...
78,花岗岩,蒙古黑,600X400X20,光面,6
79,花岗岩,蒙古黑,600X450X20,光面,4
80,花岗岩,蒙古黑,600X500X100,光面,10
81,花岗岩,蒙古黑,600X600X20,光面,1


In [ ]:
import win32com.client as win32

def extract_text_from_autocad(dxf_file_path):
    try:
        # Connect to AutoCAD
        acad = win32.Dispatch("AutoCAD.Application")
        acad.Visible = True

        # Open the DXF file
        doc = acad.Documents.Open(dxf_file_path)

        # Initialize a list to store extracted text
        extracted_text = []

        # Access all the text objects in the drawing
        for entity in doc.ModelSpace:
            if entity.EntityType == win32.constants.acText:
                extracted_text.append(entity.TextString)

        # Close the drawing without saving changes
        doc.Close(False)

        # Quit AutoCAD
        acad.Quit()

        return extracted_text

    except Exception as e:
        return str(e)

# Example usage
dxf_file_path = "path/to/your/file.dxf"
extracted_text = extract_text_from_autocad(dxf_file_path)
print(extracted_text)


In [9]:
doc = acad.ActiveDocument
for i in range(1,len(doc.Layouts)):
    print(doc.Layouts[i].Name)
    for entity in doc.Layouts[i]:
        name = entity.EntityName
        if name == 'Mtext':
            print(entity.TextString)

TypeError: 'IAcadLayouts' object is not subscriptable

In [4]:
for obj in doc.ModelSpace:
    if obj.EntityName == 'TDbSymbMultiLeader':
        print(obj.Layer)

KeyboardInterrupt: 

In [9]:
doc.layout

AttributeError: '<win32com.gen_py.AutoCAD 2019 Type Library.IAcadDocument instance at 0x2653032227984>' object has no attribute 'layout'

In [4]:
for lyt in doc.Layouts:
    for obj in lyt.Block:
        if obj.EntityName == 'TDbSymbMultiLeader':
            print(obj.UpText)

AttributeError: '<win32com.gen_py.AutoCAD 2021 Type Library.IAcadEntity instance at 0x2516087152592>' object has no attribute 'UpText'

In [31]:
for obj in doc.Blocks:
    if obj.Name == '11123':
        print(obj.x_coord)

AttributeError: '<win32com.gen_py.AutoCAD 2014 Type Library.IAcadBlock instance at 0x1705955885360>' object has no attribute 'x_coord'

In [32]:
import win32com.client as win32

wincad = win32.Dispatch("AutoCAD.Application")
version_str = wincad.Version[:4]
if version_str[3] == '0':
	progid = version_str[:2]
else:
	progid = version_str

In [33]:
progid

'19.1'

In [54]:
header = ['类型', 'X坐标', 'Y坐标', 'Z坐标']
points = [['Point', '', '', '']]
lines = [['Line', '', '', '']]
polylines = [['Polyline', '', '', '']]
circles = [['Circle', '', '', '']]
splines = [['Spline', '', '', '']]

for i in slt:
	if i.ObjectName == 'AcDbPoint':
		coor_lst = list(i.Coordinates)
		coor_lst.insert(0, '点')
		points.append(coor_lst)
	elif i.ObjectName == 'AcDbLine':
		coor_lst1 = list(i.StartPoint)
		coor_lst1.insert(0, '直线(端点)')
		coor_lst2 = list(i.EndPoint)
		coor_lst2.insert(0, '')
		lines.append(coor_lst1)
		lines.append(coor_lst2)
	elif (i.ObjectName == 'AcDbPolyline') or (i.ObjectName == 'AcDb2dPolyline'):
		if len(i.Coordinates)%3 == 0:
			coor_lst1 = list(i.Coordinates[0:3])
			coor_lst1.insert(0, '多段线(端点)')
			polylines.append(coor_lst1)
			for j in range(int(len(i.Coordinates)/3-1)):
				coor_lst2 = list(i.Coordinates[(j+1)*3:(j+1)*3+3])
				coor_lst2.insert(0, '')
				polylines.append(coor_lst2)
		else:
			coor_lst1 = list(i.Coordinates[0:2])
			coor_lst1.insert(0, '多段线(端点)')
			coor_lst1.append(0)
			polylines.append(coor_lst1)
			for j in range(int(len(i.Coordinates) / 2 - 1)):
				coor_lst2 = list(i.Coordinates[(j + 1) * 2:(j + 1) * 2 + 2])
				coor_lst2.insert(0, '')
				coor_lst2.append(0)
				polylines.append(coor_lst2)
	elif i.ObjectName == 'AcDbCircle':
		coor_lst = list(i.Center)
		coor_lst.insert(0, '圆(圆心)')
		circles.append(coor_lst)
	elif i.ObjectName == 'AcDbSpline':
		coor_lst1 = list(i.FitPoints[0:3])
		coor_lst1.insert(0, '样条曲线(拟合点)')
		splines.append(coor_lst1)
		for j in range(int(len(i.FitPoints) / 3 - 1)):
			coor_lst2 = list(i.FitPoints[(j + 1) * 3:(j + 1) * 3 + 3])
			coor_lst2.insert(0, '')
			splines.append(coor_lst2)

In [55]:
circles

[['Circle', '', '', '']]

In [63]:
try:
    doc.SelectionSets.Item("SS1").Delete()
except:
    print("Delete selection failed")

Delete selection failed


In [64]:
slt = doc.SelectionSets.Add("SS1")

In [65]:
slt

<win32com.gen_py.AutoCAD 2014 Type Library.IAcadSelectionSet instance at 0x1705950734560>

In [61]:
for obj in SS1:
    print(obj.Name)

NameError: name 'SS1' is not defined

In [66]:
SS1

NameError: name 'SS1' is not defined

In [70]:
slt.SelectOnScreen()
print("Please pick up the primitives on the screen and end with Enter" )

Please pick up the primitives on the screen and end with Enter


In [71]:
slt

<win32com.gen_py.AutoCAD 2014 Type Library.IAcadSelectionSet instance at 0x1705950734560>

In [72]:
slt[0]

TypeError: 'IAcadSelectionSet' object is not subscriptable

In [69]:
obj = slt[0]
print(obj.ObjectID)
print(slt)

Please pick up the primitives on the screen and end with Enter


TypeError: 'IAcadSelectionSet' object is not subscriptable

In [3]:
folder = r'W:\20220613 中农鹤壁项目\05-施工图纸'

In [4]:
files = os.listdir(folder)
files

['XS-04 多媒体展厅详图.bak',
 'XS-04 多媒体展厅详图.dwg',
 'XS-05-01~06 游乐互动平台.bak',
 'XS-05-01~06 游乐互动平台.dwg',
 'XS-05-07~12 游乐互动平台.bak',
 'XS-05-07~12 游乐互动平台.dwg',
 'XS-05-13~16 游乐互动平台.bak',
 'XS-05-13~16 游乐互动平台.dwg',
 'XS-06 互动水幕墙详图.bak',
 'XS-06 互动水幕墙详图.dwg',
 'XS-08 水幕观景平台.bak',
 'XS-08 水幕观景平台.dwg',
 'XS-09手工坊廊架详图.bak',
 'XS-09手工坊廊架详图.dwg',
 'XS-10-01~10 造型廊架详图.bak',
 'XS-10药卉坊廊架详图.bak',
 'XS-10药卉坊廊架详图.dwg',
 'XS-16 花舞时空建筑详图.bak',
 'XS-16 花舞时空建筑详图.dwg',
 'XS-17 花漾便利店详图.bak',
 'XS-17 花漾便利店详图.dwg',
 'XS-26-01~10 鱼菜蜀寨详图.bak',
 'XS-26-01~10 鱼菜蜀寨详图.dwg',
 '中农鹤壁项目.bak',
 '中农鹤壁项目.ds$',
 '中农鹤壁项目.dst',
 '互动水幕轴测图.jpg',
 '会签栏A2.bak',
 '会签栏A2.dwg',
 '便利店.png',
 '多媒体展厅.jpg',
 '时空建筑.png',
 '树形雕像.jpg',
 '水幕观景平台.jpg',
 '游乐互动平台意向图.jpg',
 '爬梯.png']

In [20]:
for file in files:
    if file.endswith('.dwg'):
        doc = acad.Documents.Open(os.path.join(folder, file))
        for layout in doc.blocks:
            for entity in layout:
                name = entity.EntityName
                if name == 'AcDbBlockReference':
                    HasAttributes = entity.HasAttributes
                    if HasAttributes:
                        for attrib in entity.GetAttributes():
                            print("  {}: {}".format(attrib.TagString, attrib.TextString))

  A: 1
  LD-01: LD-01
  A: 1
  LD-01: LD-01
  图纸名称: 多媒体展厅详图一
  设计阶段: 初步设计
  版本号: 01
  图纸编号: XS-04-01
  项目编号: DFHM-04-2269
  出图日期: 2022年06月
  项目名称第二行: 2号馆
  项目名称第一行: %%
  开发商第一行: %%
  开发商第二行: %%
  比例: 1:300
  子项编号: 2
  图纸名称: 多媒体展厅详图二
  设计阶段: 初步设计
  版本号: 01
  图纸编号: XS-04-02
  项目编号: DFHM-04-2269
  出图日期: 2022年06月
  项目名称第二行: 2号馆
  项目名称第一行: %%
  开发商第一行: %%
  开发商第二行: %%
  比例: 1:300
  子项编号: 2


AttributeError: Open.blocks

In [1]:
doc = acad.ActiveDocument

In [2]:
# iterate through all objects (entities) in the currently opened drawing
# and if its a BlockReference, display its attributes.

In [4]:
doc = acad.ActiveDocument
for layout in doc.blocks:
    for entity in layout:
        name = entity.EntityName
        if name == 'AcDbBlockReference':
            HasAttributes = entity.HasAttributes
            if HasAttributes:
                for attrib in entity.GetAttributes():
                    print("  {}: {}".format(attrib.TagString, attrib.TextString))

  图纸名称: 游乐互动平台详图一
  设计阶段: 初步设计
  版本号: 01
  图纸编号: XS-05-01
  项目编号: DFHM-04-2269
  出图日期: 2022年06月
  项目名称第二行: 景观施工图
  项目名称第一行: %%
  开发商第一行: %%
  开发商第二行: %%
  比例: 1:300
  子项编号: 3
  A: 1
  LD-01: XS-05-07
  A: 1
  LD-01: XS-05-03
  A: 1
  LD-01: XS-05-13
  图纸名称: 游乐互动平台详图三
  设计阶段: 初步设计
  版本号: 01
  图纸编号: XS-05-03
  项目编号: DFHM-04-2269
  出图日期: 2022年06月
  项目名称第二行: 景观施工图
  项目名称第一行: %%
  开发商第一行: %%
  开发商第二行: %%
  比例: 1:300
  子项编号: 3
  A: 1
  LD-01: XS-05-07
  图纸名称: 游乐互动平台详图二
  设计阶段: 初步设计
  版本号: 01
  图纸编号: XS-05-02
  项目编号: DFHM-04-2269
  出图日期: 2022年06月
  项目名称第二行: 景观施工图
  项目名称第一行: %%
  开发商第一行: %%
  开发商第二行: %%
  比例: 1:300
  子项编号: 3
  图纸名称: 游乐互动平台详图五
  设计阶段: 初步设计
  版本号: 01
  图纸编号: XS-05-05
  项目编号: DFHM-04-2269
  出图日期: 2022年06月
  项目名称第二行: 景观施工图
  项目名称第一行: %%
  开发商第一行: %%
  开发商第二行: %%
  比例: 1:300
  子项编号: 3
  A: 4
  LD-01: XS-05-06
  图纸名称: 游乐互动平台详图四
  设计阶段: 初步设计
  版本号: 01
  图纸编号: XS-05-04
  项目编号: DFHM-04-2269
  出图日期: 2022年06月
  项目名称第二行: 景观施工图
  项目名称第一行: %%
  开发商第一行: %%
  开发商第二行: %%
  比例: 1:300
  子项编号: 3


# 批量打开文件

In [89]:
for file in files:
    if file.endswith('.dwg'):
        print(str(os.path.join(folder,file)))
        acad.Documents.Open(os.path.join(folder,file))
        time.sleep(2)

W:\20220613 中农鹤壁项目\05-施工图纸\XS-04 多媒体展厅详图.dwg
W:\20220613 中农鹤壁项目\05-施工图纸\XS-05-01~06 游乐互动平台.dwg
W:\20220613 中农鹤壁项目\05-施工图纸\XS-05-07~12 游乐互动平台.dwg
W:\20220613 中农鹤壁项目\05-施工图纸\XS-05-13~16 游乐互动平台.dwg
W:\20220613 中农鹤壁项目\05-施工图纸\XS-06 互动水幕墙详图.dwg
W:\20220613 中农鹤壁项目\05-施工图纸\XS-08 水幕观景平台.dwg
W:\20220613 中农鹤壁项目\05-施工图纸\XS-09手工坊廊架详图.dwg
W:\20220613 中农鹤壁项目\05-施工图纸\XS-10药卉坊廊架详图.dwg
W:\20220613 中农鹤壁项目\05-施工图纸\XS-16 花舞时空建筑详图.dwg
W:\20220613 中农鹤壁项目\05-施工图纸\XS-17 花漾便利店详图.dwg
W:\20220613 中农鹤壁项目\05-施工图纸\XS-26-01~10 鱼菜蜀寨详图.dwg
W:\20220613 中农鹤壁项目\05-施工图纸\会签栏A2.dwg


In [ ]:
doc = acad.ActiveDocument
for layout in doc.Layouts:
    for entity in layout.Block:
        name = entity.EntityName
        if name == 'AcDbBlockReference':
            HasAttributes = entity.HasAttributes
            if HasAttributes:
                for attrib in entity.GetAttributes():
                    if attrib.TagString == '项目名称第二行':
                        attrib.TextString = '%<\AcSm Sheet.子项名称>%'
    time.sleep(2)

In [6]:
doc = acad.ActiveDocument
for layout in doc.Layouts:
    print(layout)
    print(len(doc.Layouts))
    # for entity in layout[1:].Block:
    #     print(entity.EntityName)

<COMObject <unknown>>
7
<COMObject <unknown>>
7
<COMObject <unknown>>
7
<COMObject <unknown>>
7
<COMObject <unknown>>
7
<COMObject <unknown>>
7
<COMObject <unknown>>
7


In [10]:
doc = acad.ActiveDocument
for i in range(1,len(doc.Layouts)):
    print(doc.Layouts[i].Name)
    for entity in doc.Layouts[i].Block:
        name = entity.EntityName
        if name == 'AcDbBlockReference':
            HasAttributes = entity.HasAttributes
            if HasAttributes:
                for attrib in entity.GetAttributes():
                    if attrib.TagString == '项目名称第二行':
                        attrib.TextString = '%<\AcSm Sheet.子项名称>%'

XS-05-01 游乐互动平台详图一
XS-05-02 游乐互动平台详图二
XS-05-03 游乐互动平台详图三
XS-05-04 游乐互动平台详图四
XS-05-05 游乐互动平台详图五
XS-05-06 游乐互动平台详图六


In [15]:
def PrintAttribute(dwg_file):
    doc = acad.Documents.Open(dwg_file)
    for i in range(1,len(doc.Layouts)):
        print(doc.Layouts[i].Name)
        for entity in doc.Layouts[i].Block:
            name = entity.EntityName
            if name == 'AcDbBlockReference':
                HasAttributes = entity.HasAttributes
                if HasAttributes:
                    for attrib in entity.GetAttributes():
                        if attrib.TagString == '项目名称第二行':
                            attrib.TextString = '%<\AcSm Sheet.子项名称>%'

In [32]:
dwg_file = []
for file in files:
    if file.endswith('.dwg'):
        dwg_file.append(os.path.join(folder,file))

In [33]:
dwg_file

['W:\\20220613 中农鹤壁项目\\05-施工图纸\\XS-04 多媒体展厅详图.dwg',
 'W:\\20220613 中农鹤壁项目\\05-施工图纸\\XS-05-01~06 游乐互动平台.dwg',
 'W:\\20220613 中农鹤壁项目\\05-施工图纸\\XS-05-07~12 游乐互动平台.dwg',
 'W:\\20220613 中农鹤壁项目\\05-施工图纸\\XS-05-13~16 游乐互动平台.dwg',
 'W:\\20220613 中农鹤壁项目\\05-施工图纸\\XS-06 互动水幕墙详图.dwg',
 'W:\\20220613 中农鹤壁项目\\05-施工图纸\\XS-08 水幕观景平台.dwg',
 'W:\\20220613 中农鹤壁项目\\05-施工图纸\\XS-09手工坊廊架详图.dwg',
 'W:\\20220613 中农鹤壁项目\\05-施工图纸\\XS-10药卉坊廊架详图.dwg',
 'W:\\20220613 中农鹤壁项目\\05-施工图纸\\XS-16 花舞时空建筑详图.dwg',
 'W:\\20220613 中农鹤壁项目\\05-施工图纸\\XS-17 花漾便利店详图.dwg',
 'W:\\20220613 中农鹤壁项目\\05-施工图纸\\XS-26-01~10 鱼菜蜀寨详图.dwg',
 'W:\\20220613 中农鹤壁项目\\05-施工图纸\\会签栏A2.dwg']

In [37]:
for file in dwg_file:
    acad.Documents.Open(file)
    time.sleep(5)
    doc = acad.ActiveDocument
    for i in range(1,len(doc.Layouts)):
        print(doc.Layouts[i].Name)
        for entity in doc.Layouts[i].Block:
            name = entity.EntityName
            if name == 'AcDbBlockReference':
                HasAttributes = entity.HasAttributes
                if HasAttributes:
                    for attrib in entity.GetAttributes():
                        if attrib.TagString == '项目编号':
                            attrib.TextString = '%<\AcSm.16.2 SheetSet.ProjectNumber>%'
                        if attrib.TagString == '比例':
                            attrib.TextString = '图示'

XS-04-01 多媒体展厅详图一
XS-04-02 多媒体展厅详图二
XS-05-01 游乐互动平台详图一
XS-05-02 游乐互动平台详图二
XS-05-03 游乐互动平台详图三
XS-05-04 游乐互动平台详图四
XS-05-05 游乐互动平台详图五
XS-05-06 游乐互动平台详图六
XS-05-07 游乐互动平台详图七
XS-05-08  游乐互动平台详图八
XS-05-09 游乐互动平台详图九
XS-05-10 游乐互动平台详图十
XS-05-11 游乐互动平台详图十一
XS-05-12 游乐互动平台详图十二
XS-05-13 游乐互动平台详图十三
XS-05-14 游乐互动平台详图十四
XS-05-15 游乐互动平台详图十五
XS-05-16 游乐互动平台详图十六
XS-06-01 互动水幕详图一
XS-06-02 互动水幕详图二
XS-06-03 互动水幕详图三
XS-08-01 水幕观景平台详图一
XS-08-02 水幕观景平台详图二
XS-08-03 水幕观景平台详图三
XS-08-04 水幕观景平台详图四
XS-08-05 水幕观景平台详图五
XS-08-06 水幕观景平台详图六
XS-08-07 水幕观景平台详图七
XS-08-08 水幕观景平台详图八
XS-08-09 水幕观景平台详图九
手工坊廊架详图一
手工坊廊架详图七
手工坊廊架详图三
手工坊廊架详图二
手工坊廊架详图五
手工坊廊架详图六
手工坊廊架详图四
XS-10-01药卉坊廊架底平面图
XS-10-02药卉坊廊架顶平面图
XS-10-03药卉坊廊架基础及龙骨布置图
XS-10-04药卉坊廊架A、B、C立面图
XS-10-05药卉坊廊架装饰大样图
XS-10-06节点详图
花舞时空建筑结构说明
花舞时空建筑详图一
花舞时空建筑详图三
花舞时空建筑详图二
花漾便利店详图一
花漾便利店详图七
花漾便利店详图三
花漾便利店详图九
花漾便利店详图二
花漾便利店详图五
花漾便利店详图八
花漾便利店详图六
花漾便利店详图十
花漾便利店详图十一
花漾便利店详图四
XS-26-01鱼菜蜀寨详图一
XS-26-02鱼菜蜀寨详图二
XS-26-03鱼菜蜀寨详图三
XS-26-04鱼菜蜀寨详图四
XS-26-05鱼菜蜀寨详图五
XS-26-06鱼菜蜀寨详图六
XS-26-07鱼菜蜀寨详图七
XS-

In [29]:
doc = acad.ActiveDocument
for i in range(1,len(doc.Layouts)):
    print(doc.Layouts[i].Name)
    for entity in doc.Layouts[i].Block:
        name = entity.EntityName
        if name == 'AcDbBlockReference':
            HasAttributes = entity.HasAttributes
            if HasAttributes:
                for attrib in entity.GetAttributes():
                    if attrib.TagString == '项目名称第二行':
                        attrib.TextString = '%<\AcSm Sheet.子项名称>%'

XS-08-01 水幕观景平台详图一
XS-08-02 水幕观景平台详图二
XS-08-03 水幕观景平台详图三
XS-08-04 水幕观景平台详图四
XS-08-05 水幕观景平台详图五
XS-08-06 水幕观景平台详图六
XS-08-07 水幕观景平台详图七
XS-08-08 水幕观景平台详图八
XS-08-09 水幕观景平台详图九
